# 描述符和 <class 'function'>

考虑：
```python
class MyDescriptor:
    def __init__(self):
        pass
    def __get__(self, instance, owner):
        return "描述符返回值"
    
def user_func(x):
    pass
```

- 含有 `__get__` 的类型称为 ***描述符***。对于一个描述符实例 `x`，`*.x` 等价为
    ```python
    x.__get__(*, type(*))
    ```

- python 中 ***自定义def*** 都是 `<class 'function'>` 类型的。该类型是由 C 层面实现的，其等价实现为
    ```python
    class Function:
    def __init__(self, func_code, func_name):
        self.__code__ = func_code
        self.__name__ = func_name

    # 绑定
    def __get__(self, instance, owner=None):
        if instance is None:
            return self
        else:
            import types
            return types.MethodType(self, instance)
    
    # 调用
    def __call__(self, *args, **kwargs):
        pass
    ```


    ```python
    print(type(MyDescriptor.__init__)) # <class 'function'>
    print(type(MyDescriptor.__get__))  # <class 'function'>
    print(type(user_func)) # <class 'function'>
    ```
    即 `<class 'function'>` 类型是描述符。

    - `<class 'function'>` 的 `__get__` 由 C 语言实现，不包含 `__get__` 即不是描述符。
        ```python
        print(type(MyDescriptor.__get__.__get__))  # <class 'method-wrapper'>
        print(type(MyDescriptor.__init__.__get__)) # <class 'method-wrapper'>
        print(type(user_func.__get__))  # <class 'method-wrapper'>
        ```

# 将 `function` 绑定为 `method`
使用 `types.MethodType` 可以将 `<class 'function'>` 变成 `<class 'method'>`。
`<class 'method'>` 的部分等价实现：
```python
class Method:
    
    def __init__(self, func, instance):
        self.__func__ = func      # 原始函数
        self.__self__ = instance  # 绑定的实例
    
    def __call__(self, *args, **kwargs):
        return self.__func__(self.__self__, *args, **kwargs)
```

例子：
```python
def standalone_function(self, x):
    return f"处理 {x} 来自 {self}"

class MyClass:
    def __init__(self, name):
        self.name = name

obj = MyClass("测试对象")

print(f"{type(standalone_function)}")  # <class 'function'>

# 绑定到实例
instance_method = types.MethodType(standalone_function, obj)
print(f"{type(instance_method)}")  # <class 'method'>

# 绑定到类
class_method = types.MethodType(standalone_function, MyClass)
print(f"{type(class_method)}")  # <class 'method'>

# 调用对比
print(f"{standalone_function(obj, 'data1')}") # 处理 data1 来自 <__main__.MyClass object at 0x00000233633C7230>
print(f"{instance_method('data2')}")  # 处理 data2 来自 <__main__.MyClass object at 0x00000233633C7230>
print(f"{class_method('data3')}")  # 处理 data3 来自 <class '__main__.MyClass'>
```

# `classmethod` 的Python等价实现
根据 Python 官方文档，`classmethod` 的纯Python等价实现是：
```python
class ClassMethod:
    def __init__(self, f):
        self.__func__ = f

    def __get__(self, obj, cls=None):
        if cls is None:
            cls = type(obj)
        # 如果 self.f 是个描述符即 function 对象，用其自身的 __get__ 绑定
        if hasattr(type(self.__func__), '__get__'):
            return self.__func__.__get__(cls, type(cls))
        return MethodType(self.__func__, cls)
```
即 `classmethod` 的核心思想当使用 `__get__` 对自身存储的 `self.__func__` 进行绑定时，一定是绑定到类而非实例上。
- 考虑
    ```python
    @classmethod
    def func():...
    ```
    等价于
    ```python 
    func = classmethod(func)    # 构建描述符对象（func 拥有了 self.__func__　属性）
    ```
- 而
    ```python
    Class/obj.func()
    ```
    等价于两步：
    ```python 
    func.__get__(None/obj, Class/type(obj)).__call__() # 先绑定再运行
    ```

# `staticmethod` 的Python等价实现
根据 Python 官方文档，`staticmethod` 的纯Python等价实现是：
```python
class staticmethod:
    def __init__(self, func):
        self.func = func
    
    def __get__(self, instance, owner):
        return self.func
```

## 代码分析：`class_or_instancemethod`
```python
class class_or_instancemethod(classmethod):
    def __get__(self, instance: object, owner: tp.Optional[tp.Type] = None) -> tp.Any:
        descr_get = super().__get__ if instance is None else self.__func__.__get__
        return descr_get(instance, owner)
```

- 考虑
    ```python
    @class_or_instancemethod
    def func():...
    ```
    等价于
    ```python 
    func = class_or_instancemethod(func) # 构建描述符对象（func 拥有了 self.__func__　属性）
    ```
- 而
    ```python
    Class/obj.func()
    ```
    等价于
    ```python 
    func.__get__(None/obj, Class/type(obj)).__call__() # 先绑定再运行
    ```
    其中 
    ```python
    func.__get__(None/obj, Class/type(obj)) # 返回的是 <class 'method'> 对象
    ```
    会根据 `instance` 是否为空来决定将 `func` 绑定到 `Class` 或 `obj` 上。



# 描述符协议

```python
class Descriptor:

    def __init__(self):
        ...

    def __get__(self, instance, owner):
        ...

    def __set__(self, instance, value):
        ...

    def __delete__(self, instance):
        ...

class User:
    descriptor = Descriptor()

user = User()
```

当类似于 `= user.descriptor` 使用时，实际上是触发了 `descriptor.__get__(user, User)`。

`user.descriptor = 值` 实际上触发了 `descriptor.__set__(user, 值)`。

`del user.descriptor` 实际上触发了 `descriptor.__delete__(user)`。

## property 类的 Python 等价实现

**注意 `porperty` 是由 C 层面实现的，所以 `__get__`、`__set__`、`__delete__` 都不是描述符！**

```python
class Property:
    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc"

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    # 返回新的 Property 实例
    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)
```

## @property

使用方法如下

```python
class MyClass:
    def __init__(self):

    # value = property(函数 value(self))
    @property
    def value(self):
        ...

    # value = value.setter(函数 value(self, new_value))
    @value.setter
    def value(self, new_value):
        ...

    # value = value.setter(函数 value(self))
    @value.deleter
    def value(self):
        ...
```

所以相当于创建了三次 property 实例，最后该实例成为 MyClass 的一个类属性 value。

然后 MyClass 或其实例就可以按照 `描述符协议` 来使用 value。

# `a.b` 的内部过程
- (1) 在类层次结构 `type(a).__mro__` 中查找是否有名为 `b` 的数据描述符。

    `数据描述符`：同时实现了 `__get__` 和 `__set__` 方法的对象。这种情况下，`a.b` 相当于 
    ```python
    b.__get__(a, type(a))
    ```
    如果 `__get__` 是自定义的即是 `<class 'function'>`，那么实际上发生了
    ```python
    __get__.__get__(b, type(b))(a, type(a))
    ```

    其中第二个 `__get__` 是 C 层面定义的，所以实际上相当于

    ```python
    __get__(__get__, b, type(b))(a, type(a)) 
    ```
- (2) 查找实例字典
    ```python
    if hasattr(a, '__dict__') and 'b' in a.__dict__:
        return a.__dict__['b']
    ```
- (3) 查找非数据描述符和类属性
    - `非数据描述符`：只实现了 `__get__` 方法（如函数、方法、property等）。

    - `普通类属性`：存储在类的 `__dict__` 中的普通值

    如果找到描述符，调用其 `__get__(a, type(a))` 方法；如果是普通属性，直接返回。
- (4) 调用 `__getattr__` 方法

    如果以上步骤都没有找到属性，且对象定义了 `__getattr__` 方法，会调用：
    ```python
    a.__getattr__('b')
    ```
- (5) 抛出 `AttributeError`

    如果所有步骤都失败，抛出 `AttributeError` 异常。